In [ ]:
pip install --upgrade undetected_chromedriver

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime
import undetected_chromedriver as uc
import re
import os

categorias=[" https://www.falabella.com.pe/falabella-pe/category/cat1470538/Zapatos",
            "https://www.falabella.com.pe/falabella-pe/category/cat6630546/Zapatos-Nino",
            "https://www.falabella.com.pe/falabella-pe/category/cat6630545/Zapatos-Nina",
            "https://www.falabella.com.pe/falabella-pe/category/cat1470534/Zapatillas-urbanas-mujer",
            "https://www.falabella.com.pe/falabella-pe/category/cat16200461/Zapatillas-deportivas-mujer",
            "https://www.falabella.com.pe/falabella-pe/category/cat6630526/Zapatos-casuales-mujer",
            "https://www.falabella.com.pe/falabella-pe/collection/zapatos-formales-mujer",
            "https://www.falabella.com.pe/falabella-pe/category/cat6630524/Botas-de-Mujer",
            "https://www.falabella.com.pe/falabella-pe/category/CATG15669/Botines-mujer",
            "https://www.falabella.com.pe/falabella-pe/category/CATG15673/Mocasines",
            "https://www.falabella.com.pe/falabella-pe/category/cat1470530/Sandalias-mujeres",
            "https://www.falabella.com.pe/falabella-pe/category/cat6630532/Zapatillas-Mujer?f.product.attribute.Tipo=Zapatillas+outdoor",
            "https://www.falabella.com.pe/falabella-pe/category/cat16200461/Zapatillas-deportivas-mujer?f.product.attribute.Tipo=Zapatillas%20running",
            " https://www.falabella.com.pe/falabella-pe/category/cat4100462/Moda-Mujer ",
            " https://www.falabella.com.pe/falabella-pe/category/CATG15668/Pijamas ",
            " https://www.falabella.com.pe/falabella-pe/collection/lenceria ",
            " https://www.falabella.com.pe/falabella-pe/category/cat7730462/Sostenes ",
            " https://www.falabella.com.pe/falabella-pe/category/cat7730463/Calzones ",
            " https://www.falabella.com.pe/falabella-pe/category/cat12980475/Fajas-y-Modeladores ",
            " https://www.falabella.com.pe/falabella-pe/category/CATG15664/Medias ",
            " https://www.falabella.com.pe/falabella-pe/category/cat7730461/Batas ",
            " https://www.falabella.com.pe/falabella-pe/category/CATG15667/Camisas-de-dormir ",
            " https://www.falabella.com.pe/falabella-pe/category/CATG15666/Pantys ",
            " https://www.falabella.com.pe/falabella-pe/category/CATG15658/Correas ",
            " https://www.falabella.com.pe/falabella-pe/category/CATG15486/Ropa-de-bebe-nino ",
            " https://www.falabella.com.pe/falabella-pe/category/cat7350564/Ropa-de-ninas ",
            " https://www.falabella.com.pe/falabella-pe/category/cat7350562/Ropa-de-ninos ",
            " https://www.falabella.com.pe/falabella-pe/category/cat7230501/Correas ",
            " https://www.falabella.com.pe/falabella-pe/category/cat4100481/Moda-Hombre ",
            " https://www.falabella.com.pe/falabella-pe/category/cat4100491/Ropa-interior-y-pijamas "
]


# Función para acceder a la página web y extraer el HTML
def acceso(url, browser):
    try:
        browser.get(url)

        # Intentar aceptar cookies si el botón existe
        try:
            WebDriverWait(browser, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="testId-accept-cookies-btn"]'))
            ).click()
        except:
            # Si no se encuentra el botón, continuar sin error
            pass

        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'html'))
        )
        html = browser.page_source
        soup = bs(html, "html.parser")
    except Exception as e:
        print(f"Error al acceder a la página {url}: {e}")
        soup = None
    return soup

# Función para obtener la cantidad de páginas que tiene una categoría
def number_page(enlace, browser):
    soup = acceso(enlace, browser)
    if soup:
        articulos = soup.find("span", {"class": "copy5 secondary jsx-3451706699 normal"}).text
        numero = int(re.search(r'\((\d+)\)', articulos).group(1)) // 48 + 1
        return min(numero, 200)  # Limitar a un máximo de 200 páginas
    return 1  # Si no se obtiene información, asumir una sola página

# Función para crear la URL de la página i
def next_page(enlace, pagina):
    return f"{enlace}?page={pagina}"

# Función para extraer texto
def text_extract(textos):
    return [texto.text for texto in textos]

# Función para extraer precios
def prices_extract(precios):
    return [precio.text.strip()[3:] for precio in precios]

# Función para combinar listas de listas en una sola lista
def unir_listas(lista_de_listas):
    return [item for sublista in lista_de_listas for item in sublista]

CSV_FILENAME = "Saga_Falabella.csv"

# Inicialización de listas globales para los datos de todas las categorías
items_totales = []
marcas_totales = []
precios_totales = []
categorias_totales = []

# Iniciar el navegador
browser = uc.Chrome()

try:
    # Si existe el CSV, lo borramos para empezar fresco
    if os.path.exists(CSV_FILENAME):
        os.remove(CSV_FILENAME)

    # Iterar sobre las categorías
    for categoria in categorias:
        n = number_page(categoria, browser)

        for i in range(1, n + 1):
            url_page = next_page(categoria, i)
            soup = acceso(url_page, browser)

            if soup:
                # Extraer los datos de los productos
                items = soup.find_all("b", {"class": "jsx-184544934 copy2 primary jsx-3451706699 normal line-clamp line-clamp-3 pod-subTitle subTitle-rebrand"})
                marcas = soup.find_all("b", {"class": "jsx-184544934 title1 secondary jsx-3451706699 bold pod-title title-rebrand"})
                precios = soup.find_all("span", {"class": "copy10 primary medium jsx-3451706699 normal line-height-22"})

                if items and marcas and precios:
                    # Crear DataFrame temporal para esta página
                    df_temp = pd.DataFrame({
                        "Item": text_extract(items),
                        "Marca": text_extract(marcas),
                        "Precio": prices_extract(precios),
                        "Categoria": [categoria.split('/')[-1]] * len(items),
                        "Web": "Falabella",
                        "Fecha": datetime.now().strftime("%m/%d/%Y")
                    })

                    # Guardar en CSV modo append
                    # Si el archivo no existe, escribir encabezados
                    file_exists = os.path.isfile(CSV_FILENAME)
                    df_temp.to_csv(CSV_FILENAME, mode='a', index=False, header=not file_exists)

            time.sleep(2)
            if i % 50 == 0:
                time.sleep(10)

finally:
    browser.quit()